In [1]:
from torch.utils.data import Dataset
import os
from PIL import Image

class SemanticSegmentationDataset(Dataset):
    """Image (semantic) segmentation dataset."""

    def __init__(self, root_dir, feature_extractor, train=True):
        """
        Args:
            root_dir (string): Root directory of the dataset containing the images + annotations.
            feature_extractor (SegFormerFeatureExtractor): feature extractor to prepare images + segmentation maps.
            train (bool): Whether to load "training" or "validation" images + annotations.
        """
        self.root_dir = root_dir
        self.feature_extractor = feature_extractor
        self.train = train

        sub_path = "training" if self.train else "validation"
        self.img_dir = os.path.join(self.root_dir, "images", sub_path)
        self.ann_dir = os.path.join(self.root_dir, "annotations", sub_path)
        
        # read images
        image_file_names = []
        for root, dirs, files in os.walk(self.img_dir):
            image_file_names.extend(files)
        self.images = sorted(image_file_names)
        
        # read annotations
        annotation_file_names = []
        for root, dirs, files in os.walk(self.ann_dir):
            annotation_file_names.extend(files)
        self.annotations = sorted(annotation_file_names)

        assert len(self.images) == len(self.annotations), "There must be as many images as there are segmentation maps"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        
        image = Image.open(os.path.join(self.img_dir, self.images[idx]))
        segmentation_map = Image.open(os.path.join(self.ann_dir, self.annotations[idx]))

        # randomly crop + pad both image and segmentation map to same size
        encoded_inputs = self.feature_extractor(image, segmentation_map, return_tensors="pt")

        for k,v in encoded_inputs.items():
            encoded_inputs[k].squeeze_() # remove batch dimension

        return encoded_inputs

In [2]:
from transformers import SegformerFeatureExtractor

# root_dir = './ADE20k_toy_dataset'
root_dir = './custom_dataset_2'


feature_extractor = SegformerFeatureExtractor(reduce_labels=True)

train_dataset = SemanticSegmentationDataset(root_dir=root_dir, feature_extractor=feature_extractor)
valid_dataset = SemanticSegmentationDataset(root_dir=root_dir, feature_extractor=feature_extractor, train=False)

/opt/conda/lib/python3.7/site-packages/transformers/models/segformer/feature_extraction_segformer.py:31: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/transformers/models/segformer/image_processing_segformer.py:105: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  FutureWarning,


In [3]:

# encoded_inputs = train_dataset[0]
# print(encoded_inputs["pixel_values"].shape)
# print(encoded_inputs["labels"].shape)
# print(encoded_inputs["labels"].squeeze().unique())

In [4]:

encoded_inputs = train_dataset[0]
     
encoded_inputs["pixel_values"].shape
     
encoded_inputs["labels"].shape
     
encoded_inputs["labels"]
     
encoded_inputs["labels"].squeeze().unique()


from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=2)

batch = next(iter(train_dataloader))
     

for k,v in batch.items():
    print(k, v.shape)
     

batch["labels"].shape

mask = (batch["labels"] != 255)
mask



batch["labels"][mask]

pixel_values torch.Size([2, 3, 512, 512])
labels torch.Size([2, 512, 512])


tensor([2, 2, 2,  ..., 7, 7, 7])

In [5]:
from transformers import SegformerForSemanticSegmentation
import json
from huggingface_hub import cached_download, hf_hub_url

# load id2label mapping from a JSON on the hub
# repo_id = "datasets/huggingface/label-files"
# filename = "ade20k-id2label.json"
filename = "./custom_dataset_2/label_json.json"
# id2label = json.load(open(cached_download(hf_hub_url(repo_id, filename)), "r"))
id2label = json.load(open(filename, "r"))
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}

# define model
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b5",
                                                         num_labels=13, 
                                                         id2label=id2label, 
                                                         label2id=label2id,
)
# model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b5",
#                                                          num_labels=150, 
#                                                          id2label=id2label, 
#                                                          label2id=label2id,
# )




from datasets import load_metric

metric = load_metric("mean_iou")

Some weights of the model checkpoint at nvidia/mit-b5 were not used when initializing SegformerForSemanticSegmentation: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_fuse.weight', 'decode_head.l

In [ ]:
import torch
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

# define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006)
# move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()
for epoch in range(1000):  # loop over the dataset multiple times
    print("Epoch:", epoch)
    for idx, batch in enumerate(tqdm(train_dataloader)):
        # get the inputs;
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits

        loss.backward()
        optimizer.step()

        # evaluate
        with torch.no_grad():
            upsampled_logits = nn.functional.interpolate(logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
            predicted = upsampled_logits.argmax(dim=1)

          # note that the metric expects predictions + labels as numpy arrays
            metric.add_batch(predictions=predicted.detach().cpu().numpy(), references=labels.detach().cpu().numpy())

        # let's print loss and metrics every 100 batches
        if idx % 100 == 0:
#             metrics = metric.compute(num_labels=len(id2label), 
#                                    ignore_index=255,
#                                    reduce_labels=False, # we've already reduced the labels before)
#             )
            metrics = metric._compute(num_labels=len(id2label),ignore_index=255, reduce_labels=False
                                      ,predictions=predicted.detach().cpu().numpy()
                                      ,references=labels.detach().cpu().numpy())

            print("Loss:", loss.item())
            print("Mean_iou:", metrics["mean_iou"])
            print("Mean accuracy:", metrics["mean_accuracy"])
    if epoch % 5 == 0:
        torch.save(model, f"./custom_checkpoint_2/model_{epoch}.pth")

Epoch: 0


  0%|          | 0/90 [00:00<?, ?it/s]

/root/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/d4add40cf977cdd73590b5873fa830f3f13adb678f6777a29fb07b7c81d14342/mean_iou.py:259: RuntimeWarning: invalid value encountered in true_divide
  acc = total_area_intersect / total_area_label


Loss: 2.490772247314453
Mean_iou: 0.03334810237939581
Mean accuracy: 0.10793612466864358
Epoch: 1


  0%|          | 0/90 [00:00<?, ?it/s]

...

Loss: 0.003824554616585374
Mean_iou: 0.9883666954243119
Mean accuracy: 0.9952255674367119
Epoch: 810


  0%|          | 0/90 [00:00<?, ?it/s]

Loss: 0.004865817725658417
Mean_iou: 0.9863380113019065
Mean accuracy: 0.9913461944598924
Epoch: 811


  0%|          | 0/90 [00:00<?, ?it/s]

Loss: 0.009378612041473389
Mean_iou: 0.9770716142528004
Mean accuracy: 0.9827407093754463
